In [5]:
# from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
import os
import json
import re
import datetime
import sys
import pandas as pd

try: 
    from pyvirtualdisplay import Display
except  ModuleNotFoundError:
    !pip install PyVirtualDisplay
    
try: 
    from seleniumwire import webdriver  # Import from seleniumwire
except  ModuleNotFoundError:
    print("module selenium-wire is not installed, now start installation.")
    !pip install selenium-wire

In [8]:
# For mouse scroll
def wheel_element(element, deltaX = 0, deltaY = 120, offsetX = 0, offsetY = 0):
    error = element._parent.execute_script("""
    var element = arguments[0];
    var deltaX = arguments[1]
    var deltaY = arguments[2];
    var box = element.getBoundingClientRect();
    var clientX = box.left + (arguments[3] || box.width / 2);
    var clientY = box.top + (arguments[4] || box.height / 2);
    var target = element.ownerDocument.elementFromPoint(clientX, clientY);

    for (var e = target; e; e = e.parentElement) {
      if (e === element) {
        target.dispatchEvent(new MouseEvent('mouseover', {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY}));
        target.dispatchEvent(new MouseEvent('mousemove', {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY}));
        target.dispatchEvent(new WheelEvent('wheel',     {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY, deltaX: deltaX, deltaY: deltaY}));
        return;
      }
    }    
    return "Element is not interactable";
    """, element, deltaX, deltaY, offsetY)

def get_websocket_data(ticker, interval, monitor=True):
    # Initiate a chrome web driver
    chrm_options = Options()
    chrm_caps = webdriver.DesiredCapabilities.CHROME.copy()
    chrm_caps['goog:loggingPrefs'] = { 'performance':'ALL' } #save all returns including websocket data
    
    if monitor == False:
        display = Display(visible=0, size=(1024, 768)) 
        display.start()
    
    driver = webdriver.Chrome(chrome_options=chrm_options,
                              desired_capabilities=chrm_caps) 

    driver.maximize_window()
    # Go to Trading View
    website = 'https://www.tradingview.com/'
    driver.get(website)
    print('Reached tradingview website')
    
    # Input query
    driver.find_element_by_tag_name('body').send_keys(Keys.COMMAND + 'k')
    driver.implicitly_wait(3)

    query = driver.find_element_by_name('query')
    query.send_keys(ticker)
    print('Typed in ticker name')
    driver.implicitly_wait(3)

#     query.submit()
    query.send_keys(Keys.ENTER)
    print('Pressed search')
    driver.implicitly_wait(3)
    
    # Select frequency
    control = driver.find_element_by_id('header-toolbar-intervals')
    control.click()
    
    if interval == "1h":
        select_1h = driver.find_element_by_xpath("//div[@data-value='60']")
        select_1h.click()
        print('Selected 1h interval')

#         driver.refresh()
        
    # Scroll the chart to retrieve all data    
    elm = driver.find_element_by_class_name('chart-gui-wrapper')
    for i in range(100):
        wheel_element(elm, deltaX=0, deltaY=100)
    print('Zoomed out')

    for i in range(1000):
        wheel_element(elm, deltaX=-10, deltaY=0)
    print('Zoomed left')
    driver.close()
    print('Done!')
    
    if monitor == False:
        display.end()
        
    return driver.requests

def parse_driver_requests(driver_requests):
    ws = []
    for request in driver_requests:
        if request.ws_messages:
            ws.append(request.ws_messages)
            
    messages = []
    for w in ws:
        for l in w:
            m = l.content
            txt_list = re.split('~m~\d*~m~',m)       
            for t in txt_list:
                if len(t)<10:
                    continue
                t = json.loads(t)
                try:
                    if t['m'] == 'timescale_update':
                        messages.append(t)
                except:
                    continue
                    
    price_data = []

    for m in messages:
        try:
            _j = m['p'][1]
            price_data.append([s['v']  for s in _j['sds_1']['s']])
        except:
            continue
            
    if len(price_data)==0:
        print('ticker not avaliable')
        return None
    df = [pd.DataFrame(p) for p in price_data]
    df = pd.concat(df)

    df.columns = ['Date', 'open', 'high', 'low', 'close', 'volume']
    df.Date = pd.to_datetime(df.Date, unit='s')
    
    print('before drop duplicate', len(df))
    df = df.drop_duplicates('Date', keep='last')
    df.sort_values('Date', inplace=True)
    print('after',len(df))
    df.reset_index(inplace=True, drop=True)
    return df

In [9]:
driver_requests = get_websocket_data('ES1!', '1h', True)
data = parse_driver_requests(driver_requests)

Reached tradingview website
Typed in ticker name
Pressed search
Selected 1h interval
Zoomed out
Zoomed left
Done!
before drop duplicate 5483
after 5262


In [10]:
data.tail(50)

,Date,open,high,low,close,volume
5212,2021-11-12 06:00:00,4650.75,4652.00,4644.75,4646.00,5778.0
5213,2021-11-12 07:00:00,4646.00,4649.75,4644.50,4646.50,6623.0
5214,2021-11-12 08:00:00,4646.75,4651.50,4644.50,4649.50,16763.0
5215,2021-11-12 09:00:00,4649.75,4650.75,4643.75,4647.00,13171.0
5216,2021-11-12 10:00:00,4647.00,4652.00,4646.25,4650.00,8848.0
5217,2021-11-12 11:00:00,4650.00,4654.75,4648.25,4654.00,8592.0
5218,2021-11-12 12:00:00,4654.00,4654.50,4650.25,4650.50,8674.0
5219,2021-11-12 13:00:00,4650.75,4656.25,4650.25,4655.00,14925.0
5220,2021-11-12 14:00:00,4655.00,4659.00,4645.75,4647.50,133322.0
5221,2021-11-12 15:00:00,4647.50,4658.50,4646.00,4658.25,147135.0


In [8]:
data.head(50)

,Date,open,high,low,close,volume
0,2021-01-28 23:00:00,36730.0,36730.0,36730.0,36730.0,0.0
1,2021-01-31 23:00:00,35700.0,35700.0,35700.0,35700.0,0.0
2,2021-02-01 23:00:00,38245.0,38245.0,38245.0,38245.0,0.0
3,2021-02-02 23:00:00,39485.0,39485.0,39485.0,39485.0,0.0
4,2021-02-03 23:00:00,39990.0,39990.0,39990.0,39990.0,0.0
5,2021-02-04 23:00:00,40300.0,40300.0,40300.0,40300.0,0.0
6,2021-02-07 23:00:00,47120.0,47120.0,47120.0,47120.0,2.0
7,2021-02-08 23:00:00,50700.0,50700.0,50700.0,50700.0,0.0
8,2021-02-09 23:00:00,47805.0,47805.0,47805.0,47805.0,0.0
9,2021-02-10 23:00:00,51500.0,51500.0,50700.0,50700.0,3.0
